<a href="https://colab.research.google.com/github/ollihansen90/Mathe-SH/blob/main/Labyrinthspiel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
from IPython.display import clear_output

class Raum():
    def __init__(self, richtungen=[0.7,0.3,0.8,0.2]):
        self.oben = richtungen[0]
        self.links = richtungen[1]
        self.unten = richtungen[2]
        self.rechts = richtungen[3]
        self.ist_ausgang = False
        self.hat_schluessel = False
        self.bild = self.male_raum()

    def __str__(self):
        return self.bild
    
    def setze_ausgang(self):
        self.ist_ausgang = True
        self.bild = self.male_raum()

    def setze_schluessel(self):
        self.hat_schluessel = True
        self.bild = self.male_raum()

    def male_raum(self):
        s = 2*"\u2588"
        w = 2*" "

        if self.oben<0.5:
            ersteZeile = 5*s
        else:
            ersteZeile = 2*s+w+2*s
        zweiteZeile = vierteZeile = s+3*w+s

        if self.ist_ausgang:
            dritteZeile = w+2*'\u2591'+w
        elif self.hat_schluessel:
            dritteZeile = w+1*'\u26BF'+w # w+1*'\U0001F5DD'+w
        else:
            dritteZeile = 3*w
        if self.links<0.5:
            dritteZeile = s+dritteZeile
        else:
            dritteZeile = w+dritteZeile
        
        if self.rechts<0.5:
            dritteZeile = dritteZeile+s
        else:
            dritteZeile = dritteZeile+w

        if self.unten<0.5:
            fuenfteZeile = 5*s
        else:
            fuenfteZeile = 2*s+w+2*s
        return ersteZeile+"\n"+zweiteZeile+"\n"+dritteZeile+"\n"+vierteZeile+"\n"+fuenfteZeile


class Karte():
    def __init__(self, hoehe, breite):
        self.raumliste = [[Raum([random.random() for i in range(4)]) for j in range(breite)] for k in range(hoehe)]
        self.schluesselpos = [2, 4] #[random.randint(0, hoehe), random.randint(0, breite)]
        self.ausgangpos = [hoehe-1, breite-1]
        raum = self.raumliste[self.ausgangpos[0]][self.ausgangpos[1]]
        raum.setze_ausgang()
        raum = self.raumliste[self.schluesselpos[0]][self.schluesselpos[1]]
        raum.setze_schluessel()

        # Schließe alle "Türen" nach oben aus der Karte raus
        for raum in self.raumliste[0]:
            raum.oben = 0.4
            raum.bild = raum.male_raum()
        
        # Schließe alle "Türen" nach unten aus der Karte raus
        for raum in self.raumliste[-1]:
            raum.unten = 0.4
            raum.bild = raum.male_raum()

        # Schließe alle "Türen" nach links und nach rechts aus der Karte raus
        for zeile in self.raumliste:
            zeile[0].links = 0.4
            zeile[0].bild = zeile[0].male_raum()
            zeile[-1].rechts = 0.4
            zeile[-1].bild = zeile[-1].male_raum()
        
        for j in range(hoehe):
            for i in range(breite-1):
                r1 = self.raumliste[j][i]
                r2 = self.raumliste[j][i+1]
                if r1.rechts+r2.links>1:
                    r1.rechts = 1
                    r2.links = 1
                else:
                    r1.rechts = 0
                    r2.links = 0
                r1.bild = r1.male_raum()
                r2.bild = r2.male_raum()
        
        for j in range(hoehe-1):
            for i in range(breite):
                r1 = self.raumliste[j][i]
                r2 = self.raumliste[j+1][i]
                if r1.unten+r2.oben>1:
                    r1.unten = 1
                    r2.oben = 1
                else:
                    r1.unten = 0
                    r2.oben = 0
                r1.bild = r1.male_raum()
                r2.bild = r2.male_raum()

    def male_karte(self):
        output = ""
        for zeile in self.raumliste:
            listeliste = [l.bild.split("\n") for l in zeile]
            for i in range(len(listeliste[0])):
                for j in range(len(listeliste)):
                    output = output+listeliste[j][i]
                output = output+"\n"
        return output

    def get_tueren(self, zeile, spalte):
        raum = self.raumliste[zeile][spalte]
        tür_oben = ""
        if not raum.oben < 0.5:
            tür_oben = 'oben'
        tür_rechts = ''
        if not raum.rechts < 0.5:
            tür_rechts = 'rechts'
        tür_unten = ''
        if not raum.unten < 0.5:
            tür_unten = 'unten'
        tür_links = ''
        if not raum.links < 0.5:
            tür_links = 'links'
        #richtungen = [raum.oben, raum.links, raum.unten, raum.rechts]
        #print(richtungen)
        richtungen = [tür_oben, tür_links, tür_unten, tür_rechts]
        richtungen_neu = []
        for r in richtungen:
            if r!="":
                richtungen_neu.append(r)
        richtungen = richtungen_neu
        #richtungen = [r for r in richtungen if r!=""]
        if len(richtungen)!=0:
            output = "Der Raum ist nach "+", ".join(richtungen)+" geöffnet."
        else:
            output = "Du bist gefangen!"
        if raum.hat_schluessel:
            output = output + " In diesem Raum liegt ein Schlüssel!"
        if raum.ist_ausgang:
            output = output + " In diesem Raum ist eine Tür!"
        return output

class Spieler():
    def __init__(self, karte):
        self.pos = [0, 0] # [zeile, spalte] [2, 3]
        self.karte = karte
        self.hat_schluessel = False
    
    def laufe(self, richtung): # WASD
        raum = self.karte.raumliste[self.pos[0]][self.pos[1]]
        richtung = richtung.upper()
        if richtung=="A" and self.pos[1]>0 and raum.links==1: # links
            self.pos[1] = self.pos[1]-1
        elif richtung=="W" and self.pos[0]>0 and raum.oben==1: # oben
            self.pos[0] -= 1
        elif richtung=="S" and self.pos[0]<len(self.karte.raumliste)-1 and raum.unten==1: # unten
            self.pos[0] += 1
        elif richtung=="D" and self.pos[1]<len(self.karte.raumliste[0])-1 and raum.rechts==1: # rechts
            self.pos[1] += 1
        else:
            print("Richtung nicht erlaubt!")
    
    def umsehen(self):
        print(self.karte.get_tueren(self.pos[0], self.pos[1]))

class Gamemaster():
    def __init__(self):
        pass

r = [random.random() for i in range(4)]
raum1 = Raum(r)
#print(raum1)
k = Karte(hoehe=4, breite=6)
print(k.schluesselpos)

#k.male_karte()  # Problem:  k.male_karte() gibt einen String zurück! Der print-
                #           Befehl lässt den String erst richtig darstellen.
print(k.male_karte())
print(k.get_tueren(2,4))
spieler = Spieler(k)

[2, 4]
████████████████████████████████████████████████████████████
██      ████      ████      ████      ████      ████      ██
██                          ████                ████      ██
██      ████      ████      ████      ████      ████      ██
████████████████████████  ██████████████████████████████████
████████████████████████  ██████████████████████████████████
██      ████      ████      ████      ████      ████      ██
██      ████      ████                ████                ██
██      ████      ████      ████      ████      ████      ██
████  ██████████████████  ████████████████████████████  ████
████  ██████████████████  ████████████████████████████  ████
██      ████      ████      ████      ████      ████      ██
██                                    ████  ⚿            ██
██      ████      ████      ████      ████      ████      ██
██████████████  ████████  ██████████████████  ██████████████
██████████████  ████████  ██████████████████  ██████████████
██      ████      

In [ ]:
print(spieler.pos)
for i in range(4):
    spieler.laufe("d")
    spieler.umsehen()
print(spieler.pos)
print(spieler.karte.raumliste[0][1].rechts)
print(2*'\u2591')

[3, 1]
Der Raum ist nach links, rechts geöffnet.
Der Raum ist nach oben, links, rechts geöffnet.
Der Raum ist nach links, rechts geöffnet.
Der Raum ist nach oben, links geöffnet. In diesem Raum ist eine Tür!
[3, 5]
1
░░


In [ ]:
while True:
    spieler.umsehen()
    eingabe = input("In welche Richtung möchtest du laufen? ")
    spieler.laufe(eingabe)
    clear_output()

Der Raum ist nach oben, unten, rechts geöffnet.


KeyboardInterrupt: ignored

In [ ]:
text1 = "Hallo\nMein Name ist Olli"
text2 = "Mir geht es heute gut.\nWie geht es dir?"

def verschmelze(str1, str2):
    output = ""
    liste1 = str1.split("\n")
    liste2 = str2.split("\n")
    for i in range(len(liste1)):
        output = output+liste1[i]+liste2[i]+"\n"
    return output

def verschmelze2(liste):
    output = ""
    listeliste = [l.split("\n") for l in liste]
    for i in range(len(listeliste[0])):
        for j in range(len(listeliste)):
            output = output+listeliste[j][i]
        output = output+"\n"
    return output

def verschmelze3(zeilenliste):
    output = ""
    for zeile in zeilenliste:
        listeliste = [l.split("\n") for l in zeile]
        for i in range(len(listeliste[0])):
            for j in range(len(listeliste)):
                output = output+listeliste[j][i]
            output = output+"\n"
    return output

#print(verschmelze(text1, text2))

r = [random.random() for i in range(4)]
raum1 = Raum(r)
r = [random.random() for i in range(4)]
raum2 = Raum(r)
raumliste = [Raum([random.random() for i in range(4)]).bild for j in range(8)]
karte = [[Raum([random.random() for i in range(4)]).bild for j in range(8)] for k in range(8)]
#print(raumliste)
#print(verschmelze(raum1.bild, raum2.bild))
#print(verschmelze2(raumliste))
print(karte)
print(verschmelze3(karte))

[['████  ████\n██      ██\n        ██\n██      ██\n████  ████', '██████████\n██      ██\n        ██\n██      ██\n████  ████', '██████████\n██      ██\n        ██\n██      ██\n██████████', '██████████\n██      ██\n██        \n██      ██\n██████████', '██████████\n██      ██\n██      ██\n██      ██\n██████████', '██████████\n██      ██\n██        \n██      ██\n██████████', '████  ████\n██      ██\n        ██\n██      ██\n████  ████', '██████████\n██      ██\n          \n██      ██\n████  ████'], ['██████████\n██      ██\n          \n██      ██\n██████████', '██████████\n██      ██\n        ██\n██      ██\n████  ████', '██████████\n██      ██\n        ██\n██      ██\n████  ████', '██████████\n██      ██\n          \n██      ██\n████  ████', '██████████\n██      ██\n██      ██\n██      ██\n██████████', '████  ████\n██      ██\n          \n██      ██\n████  ████', '██████████\n██      ██\n██      ██\n██      ██\n████  ████', '██████████\n██      ██\n          \n██      ██\n████  ████'], ['█